# Watson OpenScale and Watson Machine Learning 

This notebook should be run in a Watson Studio project, using **Default Spark Python 3.6** runtime environment. It assumes the required services have already been provisioned and will require service credentials and a Cloud API key to access the following Cloud services:
  * Watson Machine Learning
  * Watson OpenScale
  
The notebook will deploy a German Credit Risk model and then configure Watson OpenScale to subscribe the deployed model.

### Spark Validation

In [1]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark.')
    raise 

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191112170248-0001
KERNEL_ID = 455ff96e-d557-4274-a6e0-68cfa6067d85


### Dependency Setup

In [2]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client | tail -n 1

tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
botocore 1.12.82 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.7 which is incompatible.
Exception:
Traceback (most recent call last):
  File "/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/pip/_internal/cli/base_command.py", line 176, in main
    status = self.run(options, args)
  File "/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/pip/_internal/commands/install.py", line 441, in run
    options.target_dir, target_temp_dir, options.upgrade
  File "/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/pip/_internal/commands/install.py", line 492, in _handle_target_dir
    shutil.rmtree(target_item_dir)
  File "/home/spark/conda/envs/python3.6/lib/python3.6/shutil.py", line 486, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
  File "/home/spark/conda/envs/python3.6/lib/python3.6/shutil.py", line 428, in _rmtree_safe_fd
    onerror(os.rmdir, fullname, sys.exc_info())


### Service Credentials

We will be building a machine learning model below and deploying it to the Watson Machine Learning service. To interact with the service, you will need credentials for your Watson Machine Learning service instance. Copy and paste your WML credentials into the cell below.

In [3]:
WML_CREDENTIALS = {
  "apikey": "XycYiznO-hJ4nideySgm4bHOjuzaCAWuU8WNgOpWwms_",
  "iam_apikey_description": "Auto-generated for key 40fca5a1-ec33-48c6-a74e-7c8a4f06e6dd",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/93eb3babe6a441f4802534fa099afa74::serviceid:ServiceId-8c0d503c-4df3-4c11-ba95-04e628e079c4",
  "instance_id": "e3da0525-31af-4c9e-a2d2-3aad877dd536",
  "url": "https://us-south.ml.cloud.ibm.com"
}

To interact with the Watson OpenScale service, we will use the Cloud API Key to find the service instances GUID.

In [4]:
CLOUD_API_KEY = "VKrmMglnKS33gsLg9d5Tica9PwpSpA9Uf_1ZquX1dI6_"

### Model Parameters

We will use the following name for the Scikit model and the deployment to WML.

In [5]:
MODEL_NAME = "Spark German Risk Model"
DEPLOYMENT_NAME = "Spark German Risk Deployment"

## Deploy a model to Watson Machine Learning

In this section you will learn how to train Scikit-learn model and next deploy it as web-service using Watson Machine Learning service.

#### Load and explore the training data

In [6]:
import pandas as pd
from IPython.utils import io

with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv  -O german_credit_data_biased_training.csv
!ls -lh german_credit_data_biased_training.csv

spark = SparkSession.builder.getOrCreate()
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)

print('Number of records: ', str(df_data.count()))
print('Number of columns: ', len(df_data.columns))
df_data.printSchema()
df_data.head()

-rw-r--r-- 1 spark 4294967294 674K Nov 12 17:03 german_credit_data_biased_training.csv
Number of records:  5000
Number of columns:  21
root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWor

Row(CheckingStatus='0_to_200', LoanDuration=31, CreditHistory='credits_paid_to_date', LoanPurpose='other', LoanAmount=1889, ExistingSavings='100_to_500', EmploymentDuration='less_1', InstallmentPercent=3, Sex='female', OthersOnLoan='none', CurrentResidenceDuration=3, OwnsProperty='savings_insurance', Age=32, InstallmentPlans='none', Housing='own', ExistingCreditsCount=1, Job='skilled', Dependents=1, Telephone='none', ForeignWorker='yes', Risk='No Risk')

As you can see, we have 5000 records in this data set and the data contains twenty one fields. `Risk` field is the one you would like to predict using feedback data.

In [7]:
# Optional, use PixieDust to visualize the data
#import pixiedust

# display(df_data)

#### Build a pre-trained model

We could take our data and commence with data preparation to build and train the machine learning pipeline. However, we are just going to deploy a pre-trained model (https://github.com/pmservice/wml-sample-models/tree/master/spark/credit-risk)

In [8]:
with io.capture_output() as captured:
    !wget https://github.com/pmservice/wml-sample-models/blob/master/spark/credit-risk/model/credit-risk-model.tgz?raw=true  -O credit-risk-model.tgz
!ls -lh credit-risk-model.tgz

!rm -rf credit-risk-model
!mkdir credit-risk-model
!tar -xf credit-risk-model.tgz -C credit-risk-model
!ls credit-risk-model

-rw-r--r-- 1 spark 4294967294 37K Nov 12 17:03 credit-risk-model.tgz
metadata  stages


The code below imports our Random Forest Classifier which is a Spark pipeline model, setting up string indexers for the categorical features and the label column.

In [9]:
from pyspark.ml import Pipeline, Model, PipelineModel

model = PipelineModel.load("credit-risk-model")
pipeline = Pipeline( stages = model.stages )

#### Publish the model

In this section, the notebook uses the supplied Watson Machine Learning credentials to save the model (including the pipeline) to the WML instance. Previous versions of the model are removed so that the notebook can be run again, resetting all data for another demo.

In [10]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)
# wml_client.service_instance.get_details()

In [11]:
print("Models before removal of Deployment Name: ", DEPLOYMENT_NAME )
wml_client.repository.list_models()
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)

print("Models after removal of Deployment Name: ", DEPLOYMENT_NAME )
wml_client.repository.list_models()

Models before removal of Deployment Name:  Spark German Risk Deployment
------------------------------------  -----------------------  ------------------------  ---------
GUID                                  NAME                     CREATED                   FRAMEWORK
e5eed7fe-1baf-40a6-a305-79fb75dba167  Spark German Risk Model  2019-11-12T15:17:54.004Z  mllib-2.3
------------------------------------  -----------------------  ------------------------  ---------
Deleting deployment id 597073f0-3efc-4280-b970-5a70acb9dd9f
Deleting model id e5eed7fe-1baf-40a6-a305-79fb75dba167
Models after removal of Deployment Name:  Spark German Risk Deployment
----  ----  -------  ---------
GUID  NAME  CREATED  FRAMEWORK
----  ----  -------  ---------


In [12]:
import json

with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/credit-risk/meta/credit-risk-meta.json -O credit-risk-meta.json
!ls -lh credit-risk-meta.json
with open('credit-risk-meta.json') as f:
    [training_data_reference, *_] = json.load(f)['model_meta']['training_data_reference']
with open('credit-risk-meta.json') as f2:
    [metrics, *_] = json.load(f2)['model_meta']['evaluation']['metrics']

model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.TRAINING_DATA_REFERENCE: training_data_reference,
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": metrics['value'],
           "threshold": 0.7
        }
    ]
}

-rw-r--r-- 1 spark 4294967294 14K Nov 12 17:04 credit-risk-meta.json


In [13]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")    
    published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=df_data, pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")
    
wml_client.repository.list_models()
print("Model ID: ", model_uid)

Storing model ...
Done
------------------------------------  -----------------------  ------------------------  ---------
GUID                                  NAME                     CREATED                   FRAMEWORK
0e47d7a8-6157-48b0-ac8a-71f05aa9cceb  Spark German Risk Model  2019-11-12T17:04:51.177Z  mllib-2.3
------------------------------------  -----------------------  ------------------------  ---------
Model ID:  0e47d7a8-6157-48b0-ac8a-71f05aa9cceb


#### Deploy the model
The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [14]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']        
        scoring_url = deployment_in['entity']['scoring_url']
        break

if deployment_uid is None:
    print("Deploying model...")
    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    scoring_url = wml_client.deployments.get_scoring_url(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))
print("Scoring URL: {}".format(scoring_url))

Deploying model...


#######################################################################################

Synchronous deployment creation for uid: '0e47d7a8-6157-48b0-ac8a-71f05aa9cceb' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS.
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='95ab5f08-df78-4432-8f3a-d0835a875832'
------------------------------------------------------------------------------------------------


Model id: 0e47d7a8-6157-48b0-ac8a-71f05aa9cceb
Deployment id: 95ab5f08-df78-4432-8f3a-d0835a875832
Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/e3da0525-31af-4c9e-a2d2-3aad877dd536/deployments/95ab5f08-df78-4432-8f3a-d0835a875832/online


#### Test the scoring endpoint
We can test the deployed model endpoint with some sample data.

In [15]:
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                  "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                  "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                  "Telephone", "ForeignWorker"]
values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"]
        ]

scoring_payload = {"fields": fields, "values": values}
predictions = wml_client.deployments.score(scoring_url, scoring_payload)
print('Scoring result:', '\n fields:', predictions['fields'], '\n values: \n ', '\n  '.join([str(elem) for elem in predictions['values']]))

Scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values: 
  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, [21, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19, 20], [1.0, 1.0, 1.0, 13.0, 1343.0, 

## Configure Watson OpenScale



#### Get Watson OpenScale GUID

Each instance of OpenScale has a unique ID. We can get this value using the Cloud API key specified at the beginning of the notebook.


In [16]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.utils import get_instance_guid

WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print("Watson OpenScale GUID: {}".format(WOS_GUID))

wos_client = APIClient(aios_credentials=WOS_CREDENTIALS)
print("Watson OpenScale Python Client Version: {}".format(wos_client.version))

Watson OpenScale GUID: 60143291-0ba2-4347-950b-9038fde2a8f8
Watson OpenScale Python Client Version: 2.1.17


#### Create schema and datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL can be used to create a datamart for OpenScale.

This notebook will use the free, internal lite database. If you have previously configured OpenScale, it will use your existing datamart, and not interfere with any models you are currently monitoring. Do not update the cell below.

If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service. If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the KEEP_MY_INTERNAL_POSTGRES variable below to False below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. NO DATA MIGRATION WILL OCCUR.

Prior instances of the German Credit model will be removed from OpenScale monitoring.


In [17]:
KEEP_MY_INTERNAL_POSTGRES = True

In [18]:
try:
    data_mart_details = wos_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            print('Resetting internal datamart')
            wos_client.data_mart.delete(force=True)
            wos_client.data_mart.setup(internal_db=True)
    else:
        print('Using existing external datamart')
except:
    print('Setting up internal datamart')
    wos_client.data_mart.setup(internal_db=True)

Using existing internal datamart.


In [19]:
data_mart_details = wos_client.data_mart.get_details()
data_mart_details

{'database_configuration': {},
 'internal_database': True,
 'internal_database_pool': 'icd-psql',
 'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/93eb3babe6a441f4802534fa099afa74:60143291-0ba2-4347-950b-9038fde2a8f8::',
 'status': {'state': 'active'}}

#### Bind machine learning engines

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If this binding already exists, this code will output a warning message and use the existing binding.

Note: You can bind more than one engine instance if needed by calling ai_client.data_mart.bindings.add method. Next, you can refer to particular binding using binding_uid.

In [20]:
from ibm_ai_openscale.engines import *

binding_uid = wos_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(WML_CREDENTIALS))
if binding_uid is None:
    binding_uid = wos_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = wos_client.data_mart.bindings.get_details()
print("Model Serve Engine Binding ID: {}".format(binding_uid))

Warning during bind instance.
Status code: 409, body: {"errors":[{"code":"AIQCS0010W","message":"Service Binding with this id is already defined"}],"trace":"MDczNzRiN2QtNzE2NC00ODY5LWI5MmYtMTc4NmQ5NWJhNDVh"}
Model Serve Engine Binding ID: e3da0525-31af-4c9e-a2d2-3aad877dd536


In [21]:
wos_client.data_mart.bindings.list()
wos_client.data_mart.bindings.list_assets()

e3da0525-31af-4c9e-a2d2-3aad877dd536,WML instance,watson_machine_learning,2019-11-12T15:18:45.636Z


0e47d7a8-6157-48b0-ac8a-71f05aa9cceb,Spark German Risk Model,2019-11-12T17:05:03.457Z,model,mllib-2.3,e3da0525-31af-4c9e-a2d2-3aad877dd536,False


#### Subscriptions

Next we will subscribe OpenScale to the deployed model so that we can configure our monitors. We will first remove previous subscriptions to the German Credit model to refresh the monitors with the new model and new data. Then we will create the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [22]:
subscriptions_uids = wos_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = wos_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        wos_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

In [23]:
from ibm_ai_openscale.supporting_classes.enums import *

subscription = wos_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='Risk',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = wos_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if wos_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = wos_client.data_mart.subscriptions.get(sub)

In [24]:
subscriptions_uids = wos_client.data_mart.subscriptions.get_uids()
subscription_details = subscription.get_details()
#print(subscriptions_uids)
#print(subscription_details)

wos_client.data_mart.subscriptions.list()
wos_client.data_mart.bindings.list_assets()

1b906143-6525-47dc-9b41-ea6ac66fd1c1,Spark German Risk Model,model,e3da0525-31af-4c9e-a2d2-3aad877dd536,2019-11-12T17:05:29.583Z


0e47d7a8-6157-48b0-ac8a-71f05aa9cceb,Spark German Risk Model,2019-11-12T17:05:03.457Z,model,mllib-2.3,e3da0525-31af-4c9e-a2d2-3aad877dd536,True


#### Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model.

In [25]:
import datetime
import time

time.sleep(10)

fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)

print('Single record scoring result:', '\n fields:', scoring_response['fields'], '\n values: ', scoring_response['values'][0])

Single record scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, [21, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19, 20], [1.0, 1.0, 1.0, 13

In [26]:
# Payload will be logged in the datamart. Re-run this cell if there are no results in the table.
print('Number of records in payload table (should be 8): ', subscription.payload_logging.get_records_count())
subscription.payload_logging.show_table()

Number of records in payload table (should be 8):  0


## Next steps

OpenScale is ready to monitor the model. We can configure various types of monitors (quality, fairness, explainability, drift, etc) using either the UI or through a python client. 

__Return to the workshop instruction book.__

  
## Credits

This notebook was adapted from the following sources:
* [Monitor Models Code Pattern](https://github.com/IBM/monitor-wml-model-with-watson-openscale)
* [OpenScale Labs](https://github.com/pmservice/OpenScale-Labs)
* [OpenScale Tutorials](https://github.com/pmservice/ai-openscale-tutorials)

#### Original Authors
* Eric Martens, is a technical specialist having expertise in analysis and description of business processes, and their translation into functional and non-functional IT requirements. He acts as the interpreter between the worlds of IT and business.
* Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.